In [18]:
import pytz
from datetime import datetime
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd
from requests.adapters import HTTPAdapter, Retry

# Headers to mimic a browser visit
headers = {
    "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
    "accept-language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
    "cache-control": "no-cache",
    "pragma": "no-cache",
    "sec-ch-ua": "\"Not A(Brand\";v=\"99\", \"Google Chrome\";v=\"121\", \"Chromium\";v=\"121\"",
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "\"macOS\"",
    "sec-fetch-dest": "document",
    "sec-fetch-mode": "navigate",
    "sec-fetch-site": "cross-site",
    "sec-fetch-user": "?1",
    "upgrade-insecure-requests": "1",
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36"
}

requests.adapters.DEFAULT_POOLSIZE = 100
retries = Retry(total=20)

session = requests.Session()
session.mount('https://', requests.adapters.HTTPAdapter(pool_connections=100, pool_maxsize=100, max_retries=retries))


# This is a placeholder for the user to fill with specific parsing instructions,
# such as finding a table, list items, etc., depending on the website's structure.
# For example, to find all 'a' tags: soup.find_all('a')


In [19]:
import time

link_list = []
for i in range(63):
    # URL of the page to be scraped
    url = "https://www.anugafoodtec.com/anuga-foodtec-exhibitors/list-of-exhibitors/?route=aussteller/blaettern&fw_ajax=1&paginatevalues={%22stichwort%22:%22%22,%22suchart%22:%22alle%22}&start=" + str(0*20)
    # Sending a request to the URL
    response = session.get(url, headers=headers)

    # Parsing the page with BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')

    div_list = soup.find_all('div', class_='col col1ergebnis')

    for div in div_list:
        a = div.find('a')
        if a:
            print(a.get('title'), a.get('href'))
            link_list.append({"name": a.get('title'), "sub_link": a.get('href')})


    if i > 3:
        break
    time.sleep(0.1)


Albert Handtmann Maschinenfabrik GmbH & Co. KG   /exhibitor/handtmann/
Klinge Corporation    /exhibitor/klinge_corporation/
REDA S.p.A.    /exhibitor/reda/
WATERLEAU    /exhibitor/waterleau/
Landwehr Service GmbH    /exhibitor/_landwehrgmbh/
2M Carnitech    /exhibitor/2m_carnitech/
3-A Sanitary Standards, Inc.    /exhibitor/3-a_sanitary_standar/
4 Life S.r.l.    /exhibitor/4_life_srl/
A-LIMA-BIS SP. Z O.O.    /exhibitor/a-lima/
ABI B.V.    /exhibitor/abi/
Abl Srl.    /exhibitor/abl_srl/
Admix Europe ApS    /exhibitor/admix_europe/
AERA PACKAGING    /exhibitor/aera_packaging/
AETNA Group SpA    /exhibitor/aetna_group/
AF Food Technology GmbH & Co. KG    /exhibitor/af_food_technology_g/
AFT microwave GmbH    /exhibitor/aft_microwave_gmbh/
agrathaer GmbH    /exhibitor/agrathaer_gmbh/
Agriflex Srl    /exhibitor/agriflex/
Agrosingularity S.L.    /exhibitor/agrosingularity_sl/
AGROTEC EUROPA    /exhibitor/agrotec_europa/
AICON X-RAY GmbH    /exhibitor/aicon_x-ray_gmbh/
Air Quality Process   

In [142]:
# 깊이를 유지하며 텍스트 추출 함수
def extract_texts_with_depth(data, depth=0):
    result = []
    # 리스트 또는 딕셔너리 데이터 타입에 따라 처리
    if isinstance(data, dict):
        for key, value in data.items():
            if isinstance(value, dict) and 'text' in value:
                result.append((' ' * depth + value['text'], depth))
                if value.get('children'):
                    result.extend(extract_texts_with_depth(value['children'], depth + 1))
    elif isinstance(data, list):
        for item in data:
            result.append((' ' * depth + item['text'], depth))
            if item.get('children'):
                result.extend(extract_texts_with_depth(item['children'], depth + 1))
    return result

In [62]:
import requests
from urllib.parse import urlencode, parse_qs, urlsplit, urlunsplit

def fetch_pages(base_url, start_page=1, end_page=63, step=20):
    session = requests.Session()  # 세션을 사용하여 연결을 유지합니다.
    
    for page in range(start_page, end_page + 1):
        start = (page - 1) * step
        # URL의 쿼리 스트링을 분석하여 'start' 파라미터를 업데이트합니다.
        url_parts = list(urlsplit(base_url))
        query = dict(parse_qs(url_parts[3]))
        query['start'] = [str(start)]  # 'start' 파라미터 업데이트
        url_parts[3] = urlencode(query, doseq=True)
        page_url = urlunsplit(url_parts)
        
        response = session.get(page_url)  # 세션을 사용하여 페이지 요청
        yield response.content  # 페이지의 내용을 반환합니다.

        # 실제 코드에서는 여기에서 페이지의 HTML을 처리할 수 있습니다.
        # 예: BeautifulSoup을 사용하여 HTML을 파싱하고 필요한 데이터를 추출합니다.

# 기본 URL 설정 (첫 페이지의 URL)
base_url = "https://www.anugafoodtec.com/anuga-foodtec-exhibitors/list-of-exhibitors/?route=aussteller/blaettern&fw_ajax=1&paginatevalues={%22stichwort%22:%22%22,%22suchart%22:%22alle%22}&start=0&dat=451523"

# #link_list = []
# for page_content in fetch_pages(base_url, 22):
#     soup = BeautifulSoup(page_content, 'html.parser')
#     div_list = soup.find_all('div', class_='col col1ergebnis')

#     for div in div_list:
#         a = div.find('a')
#         if a:
#             print(a.get('title'), a.get('href'))
#             link_list.append({"name": a.get('title'), "sub_link": a.get('href')})
#     time.sleep(0.1)
#link_list = []
start_index = 59
for i, page_content in enumerate(fetch_pages(base_url, start_index + 1)):
    with open(f'./html/anugafoodtec_list_{("00" + str(i + start_index))[-2:]}', 'wb+') as f:
        f.write(page_content)




In [64]:
new_list = []

for i in range(63):
    with open(f'./html/anugafoodtec_list_{("00" + str(i))[-2:]}', 'rb') as f:  
        soup = BeautifulSoup(f.read(), 'html.parser')
        div_list = soup.find_all('div', class_='col col1ergebnis')

        for div in div_list:
            a = div.find('a')
            if a:
                new_list.append({"name": a.get('title'), "sub_link": a.get('href')})



In [85]:

from urllib.parse import urljoin
domain = 'https://www.anugafoodtec.com/'



import requests
from urllib.parse import urlencode, parse_qs, urlsplit, urlunsplit

start = 0
session = requests.Session()  # 세션을 사용하여 연결을 유지합니다.

for i in range(start, len(new_list)):
    info = new_list[i]
    name = info['name']
    sub_link = info['sub_link']

    page_url = urljoin(domain, sub_link)
    response = session.get(page_url, headers=headers)  # 세션을 사용하여 페이지 요청
    if response.status_code == 200:
        with open('./html/anugafoodtec_details/' + name.replace('/', '_'), 'wb+') as f:
            f.write(response.content)
    else:
        print(name, response.status_code)


In [82]:

check_list = [ 'BOCCARD SA'
, 'C.S.F. INOX S.p.A.'
, 'Endress+Hauser (Deutschland) GmbH+Co. KG'
, 'Dr. Harnisch Verlags GmbH'
, 'Inox-Fer s.r.l. '
, 'Krohne Messtechnik GmbH '
, 'Mayekawa Deutschland GmbH '
, 'Ningbo D & R Machinery Co., Ltd '
, 'Polyketting B. V. '
, 'Promarsa GmbH '
, 'stela Laxhuber GmbH '
, 'Troeger GmbH '
, 'XIAMEN JIAN AN FENG PACKING CO.,LTD ']

check_list = set([chk.strip() for chk in check_list])
check_url_list = []
for i, item in enumerate(new_list):
    if item['name'].strip() in check_list:
        check_url_list.append(item)
        print(i, item)
        

123 {'name': 'BOCCARD SA   ', 'sub_link': '/exhibitor/boccard/'}
209 {'name': 'C.S.F. INOX S.p.A.   ', 'sub_link': '/exhibitor/csf_inox/'}
295 {'name': 'Endress+Hauser (Deutschland) GmbH+Co. KG   ', 'sub_link': '/exhibitor/endress/'}
459 {'name': 'Dr. Harnisch Verlags GmbH   ', 'sub_link': '/exhibitor/harnisch/'}
545 {'name': 'Inox-Fer s.r.l.   ', 'sub_link': '/exhibitor/inox-fer/'}
631 {'name': 'Krohne Messtechnik GmbH   ', 'sub_link': '/exhibitor/krohne_messtechnik/'}
711 {'name': 'Mayekawa Deutschland GmbH   ', 'sub_link': '/exhibitor/mayekawa_deutschland/'}
798 {'name': 'Ningbo D & R Machinery Co., Ltd   ', 'sub_link': '/exhibitor/ningbo/'}
878 {'name': 'Polyketting B. V.   ', 'sub_link': '/exhibitor/polyketting/'}
894 {'name': 'Promarsa GmbH   ', 'sub_link': '/exhibitor/promarsa_gmbh/'}
1052 {'name': 'stela Laxhuber GmbH   ', 'sub_link': '/exhibitor/laxhuberstela/'}
1140 {'name': 'Troeger GmbH   ', 'sub_link': '/exhibitor/troeger_gmbh/'}
1227 {'name': 'XIAMEN JIAN AN FENG PACKING 

In [220]:
import os
path = './html/anugafoodtec_details'
# 디렉토리 내의 모든 파일과 디렉토리 리스트 가져오기
entries = os.listdir(path)
# 파일만 추출하기
file_list = [os.path.join(path, entry) for entry in entries if os.path.isfile(os.path.join(path, entry))]

print(file_list)

['./html/anugafoodtec_details/Handtmann Armaturenfabrik GmbH & Co. KG  ', './html/anugafoodtec_details/Sg Freelance SL   ', './html/anugafoodtec_details/Boss Verpackungsmaschinen GmbH & Co. KG  ', './html/anugafoodtec_details/VDL Packaging bv   ', './html/anugafoodtec_details/Kruse + Sohn AG   ', './html/anugafoodtec_details/Food Industry Systems GmbH   ', './html/anugafoodtec_details/LEWA GmbH   ', './html/anugafoodtec_details/EGLI AG Process Technology  ', './html/anugafoodtec_details/FOOD TECH Michael Jäger GmbH  ', './html/anugafoodtec_details/QUAT2RO GmbH & Co. KG   ', './html/anugafoodtec_details/JBT Alco-food-machines GmbH   ', './html/anugafoodtec_details/J. Rettenmaier & Söhne GmbH + Co. KG  ', './html/anugafoodtec_details/Turkoz Makina Sanayi ve tic. a.s.   ', './html/anugafoodtec_details/Servi Doryl SAS   ', './html/anugafoodtec_details/VERIPACK   ', './html/anugafoodtec_details/PiGo srl   ', './html/anugafoodtec_details/KHS GmbH   ', './html/anugafoodtec_details/F.LLI Cuomo

In [222]:
import json

def soup_to_item(soup):
    item = {}
    if soup.find('h1', class_="hl_2"):
        item["name"] = soup.find('h1', class_="hl_2").text.strip()

    if soup.find('div', class_="location-info"):
        item["address"] = soup.find('div', class_="location-info").text.replace('\t', '').replace('\n', '').strip()

    sub_div = soup.find('div', class_="text-right")
    if sub_div:
        for div in sub_div.find_all('div'):
            # 클래스명에서 'ico_' 다음에 오는 부분 추출
            key = div['class'][1][4:]
            # div의 텍스트 추출
            text = div.get_text(strip=True)
            # 결과 딕셔너리에 저장
            item[key] = text

    parent = soup.find('div', class_='xasdb-cap-products-list-entry')
    if parent:
        if parent.find('ul', class_="level-1"):
            j_data = json.loads(parent.find('ul', class_="level-1").text)
            item['product_groups'] = extract_texts_with_depth(j_data)

    return item

In [225]:
sub_list = []
fail_list = []

In [216]:
start = 0
for i in range(start, len(file_list)):
    detail = file_list[i]
    with open(detail, 'rt') as f:
        contents = f.read()
        if '<h1 class="hl_2">' in contents:
            sub_list.append(detail.split('/')[-1])
        else:
            fail_list.append(detail)
            

In [226]:
start = 0
for i in range(start, len(file_list)):
    detail = file_list[i]
    with open(detail, 'rt') as f:
        soup = BeautifulSoup(f.read(), 'html.parser')
        sub_list.append(soup_to_item(soup))
        

In [ ]:
import pandas as pd
df = pd.DataFrame(sub_list)


In [229]:
df.to_csv('./output/anugafoodtec_20240220.csv', index=False, quoting=1)

In [214]:
from urllib.parse import urljoin
domain = 'https://www.anugafoodtec.com/'
import requests
from urllib.parse import urlencode, parse_qs, urlsplit, urlunsplit

start = 0
session = requests.Session()  # 세션을 사용하여 연결을 유지합니다.


def get_page(url):
    global session 
    
    RETRY_COUNT = 3
    count = 0
    while True:
        response = session.get(url, headers=headers)  # 세션을 사용하여 페이지 요청
        if response.status_code == 200:
            if '<h1 class="hl_2">' in response.text:
                return response.content
        
        time.sleep(5 + count)
        session = requests.Session()
        count += 1

        if count >= RETRY_COUNT:
            raise Exception("호출 결과에 이상 발견!!")
    return None


for i in range(start, len(new_list)):
    info = new_list[i]
    name = info['name']
    sub_link = info['sub_link']

    file_path = './html/anugafoodtec_details/' + name.replace('/', '_')
    if file_path in fail_list:
        page_url = urljoin(domain, sub_link)
        try:
            content = get_page(page_url)
            if content:
                with open('./html/anugafoodtec_details/' + name.replace('/', '_'), 'wb+') as f:
                    f.write(content)
        except Exception as e:
            second_faile_list.append(name)

    if (i % 50) == (50-1):
        print(i)
    

49
99
149
199
249
299
349
399
449
499
549
599
649
699
749
799
849
899
949
999
1049
1099
1149
1199


In [180]:
'<h1 class="hl_2">' in 'aaaa<h1 class="hl_2">aaa'

True

1236

In [113]:
with open(file_list[139], 'rb') as f:
    print(file_list[139])
    soup = BeautifulSoup(f.read(), 'html.parser')
# div_list = soup.find_all('div', class_='col col1ergebnis')

# for div in div_list:
#     a = div.find('a')
#     if a:
#         new_list.append({"name": a.get('title'), "sub_link": a.get('href')})

./html/anugafoodtec_details/Albert Handtmann Maschinenfabrik GmbH & Co. KG  


In [135]:

from bs4 import BeautifulSoup, NavigableString, Tag, Comment, Doctype, ProcessingInstruction
import json

# accordeonlist 클래스를 가진 요소 찾기
parent = soup.find('div', class_='xasdb-cap-products-list-entry')
j_data = json.loads(parent.find('ul', class_="level-1").text)
# for element in parent.descendants:  # body 태그의 자식 노드 순회
#     if isinstance(element, NavigableString) and not isinstance(element, (Comment, Doctype, ProcessingInstruction)):
#         txt = element.strip().replace('\n\n\n', ' ')
#         parent = element.parent
#         if txt and (parent and parent.name not in ['script', 'style', 'meta', 'noscript']):
#             print(txt)
        
#     elif isinstance(element, Tag):
#         if element.name == 'div':
#             #print(element.text)
#             pass


In [ ]:
# 깊이를 유지하며 텍스트 추출 함수
def extract_texts_with_depth(data, depth=0):
    result = []
    # 리스트 또는 딕셔너리 데이터 타입에 따라 처리
    if isinstance(data, dict):
        for key, value in data.items():
            if isinstance(value, dict) and 'text' in value:
                result.append((' ' * depth + value['text'], depth))
                if value.get('children'):
                    result.extend(extract_texts_with_depth(value['children'], depth + 1))
    elif isinstance(data, list):
        for item in data:
            result.append((' ' * depth + item['text'], depth))
            if item.get('children'):
                result.extend(extract_texts_with_depth(item['children'], depth + 1))
    return result

# 추출 실행
extracted_texts = extract_texts_with_depth(j_data)

# 결과 출력
for text, depth in extracted_texts:
    print(f"Depth {depth}: {text}")

extracted_texts